In [7]:
#!wget --no-check-certificate \
 #   https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
  #  -O ./cats_and_dogs_filtered.zip

In [22]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.0.0'

In [23]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [24]:
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")

In [25]:
zip_object.extractall("./")

In [26]:
dataset_path_new = "./cats_and_dogs_filtered/"

In [27]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

In [28]:
IMG_SHAPE = (128, 128, 3)

In [29]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

In [30]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

In [31]:
base_model.trainable = False

In [32]:
base_model.output

<tf.Tensor 'out_relu_1/Identity:0' shape=(None, 4, 4, 1280) dtype=float32>

In [33]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [34]:
global_average_layer

<tf.Tensor 'global_average_pooling2d_1/Identity:0' shape=(None, 1280) dtype=float32>

In [35]:
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

In [36]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [37]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
____________________________________________________________________________________________

In [38]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [39]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [40]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.


In [41]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 1000 images belonging to 2 classes.


In [42]:
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - 208s 13s/step - loss: 0.6542 - accuracy: 0.6325 - val_loss: 0.5196 - val_accuracy: 0.7350
Epoch 2/5
16/16 [==============================] - 210s 13s/step - loss: 0.5970 - accuracy: 0.6850 - val_loss: 0.4672 - val_accuracy: 0.7820
Epoch 3/5
16/16 [==============================] - 175s 11s/step - loss: 0.5519 - accuracy: 0.7245 - val_loss: 0.4353 - val_accuracy: 0.8080
Epoch 4/5
16/16 [==============================] - 164s 10s/step - loss: 0.5129 - accuracy: 0.7595 - val_loss: 0.3939 - val_accuracy: 0.8310
Epoch 5/5
16/16 [==============================] - 175s 11s/step - loss: 0.4804 - accuracy: 0.7815 - val_loss: 0.3753 - val_accuracy: 0.8410


In [43]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [44]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.8410000205039978


In [45]:
base_model.trainable = True

In [46]:
print("Number of layersin the base model: {}".format(len(base_model.layers)))

Number of layersin the base model: 155


In [47]:
fine_tune_at = 100

In [48]:
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

In [49]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [50]:
model.fit_generator(train_generator,  
                    epochs=5, 
                    validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - 207s 13s/step - loss: 0.1657 - accuracy: 0.9395 - val_loss: 0.0793 - val_accuracy: 0.9710
Epoch 2/5
16/16 [==============================] - 194s 12s/step - loss: 0.0240 - accuracy: 0.9980 - val_loss: 0.0880 - val_accuracy: 0.9710
Epoch 3/5
16/16 [==============================] - 172s 11s/step - loss: 0.0102 - accuracy: 1.0000 - val_loss: 0.1169 - val_accuracy: 0.9650
Epoch 4/5
16/16 [==============================] - 174s 11s/step - loss: 0.0040 - accuracy: 1.0000 - val_loss: 0.1094 - val_accuracy: 0.9690
Epoch 5/5
16/16 [==============================] - 189s 12s/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.1379 - val_accuracy: 0.9640


In [51]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [52]:
print("Validation accuracy after fine tuning: {}".format(valid_accuracy))

Validation accuracy after fine tuning: 0.9639999866485596
